In [4]:
from kafka import KafkaConsumer
from tensorflow import keras
from joblib import load
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction import FeatureHasher
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import mutual_info_classif as mi
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_validate
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import make_scorer, f1_score
import river
from river import compose
from river import evaluate
from river import metrics
from river import preprocessing
from river import stream
import warnings
warnings.filterwarnings("ignore")

In [5]:
consumer = KafkaConsumer(
    'ml-raw-dns',
    bootstrap_servers="kafka:9092",
    auto_offset_reset='earliest',
    enable_auto_commit=False
)

In [6]:
# Load your initial Keras model
static_model = load('xgb.joblib')

In [7]:
# Clone the static model
dynamic_model = static_model

In [8]:
# Set the desired window size
window_size = 1000

In [23]:
# Initialize variables for window processing
window_data = []
window_size = 1000
performance_static = []
performance_dynamic = []

In [24]:
def deserialize(data):
    
    # Decode byte string and split by comma
    decoded_data = data.decode('utf-8').strip().split(',')
    
    # Remove additional characters from the 'Target Attack' field
    target_attack = decoded_data[15].strip().strip('\\n"')
    
    
    # Convert to appropriate data types
    data_dict = {
        'timestamp': decoded_data[0].strip('"'),
        'FQDN_count': int(decoded_data[1]),
        'subdomain_length': int(decoded_data[2]),
        'upper': int(decoded_data[3]),
        'lower': int(decoded_data[4]),
        'numeric': int(decoded_data[5]),
        'entropy': float(decoded_data[6]),
        'special': int(decoded_data[7]),
        'labels': int(decoded_data[8]),
        'labels_max': int(decoded_data[9]),
        'labels_average': float(decoded_data[10]),
        'longest_word': decoded_data[11],
        'sld': decoded_data[12],
        'len': int(decoded_data[13]),
        'subdomain': int(decoded_data[14]),
        'Target Attack': int(target_attack)
    }
    
    df = pd.DataFrame([data_dict])

    return df

In [25]:
def impute_with_max(df):
    for column in df.columns:
        # Check if the column is numerical
        if pd.api.types.is_numeric_dtype(df[column]):
            max_value = df[column].max()
            df[column].fillna(max_value, inplace=True)
        else:
            # For non-numeric columns, consider using mode (most frequent value) or another method
            most_common = df[column].mode().iloc[0]
            df[column].fillna(most_common, inplace=True)

    return df

In [26]:
def preprocess(df):
    df = df.drop(columns='timestamp')
    
    df = impute_with_max(df)
    
    types = df.dtypes
    #Indexes of categorical columns
    categorical_indexes = (types == 'object')
    #Indexes of numerical columns
    numerical_indexes = ~categorical_indexes
    
    columns = df.columns
    numerical_columns = columns[numerical_indexes]
    categorical_columns = columns[categorical_indexes]
    
    # For outliers, we'll cap the values at the 1st and 99th percentiles for each numerical feature
    for feature in numerical_columns:
        # Calculate the 1st and 99th percentiles
        lower_bound, upper_bound = df[feature].quantile([0.01, 0.99]).values
        # Cap the outliers
        df[feature] = np.clip(df[feature], lower_bound, upper_bound)
        
    # Length of the second-level domain
    df['sld_length'] = df['sld'].apply(len)
    # Presence of numeric characters in the second-level domain
    df['sld_numeric'] = df['sld'].str.contains('\d').astype(int)
    
    # Length of the longest word
    df['longest_word_length'] = df['longest_word'].apply(len)
    # Presence of numeric characters in the longest word
    df['longest_word_numeric'] = df['longest_word'].str.contains('\d').astype(int)
    
    n_features = 2**4
    # Initialize the FeatureHasher
    hasher_longest_word = FeatureHasher(n_features=n_features, input_type='string')
    hasher_sld = FeatureHasher(n_features=n_features, input_type='string')
    words_longest_word = df['longest_word'].astype(str).map(lambda x: x.split())
    words_sld = df['sld'].astype(str).map(lambda x: x.split())
    hashed_features_longest_word = hasher_longest_word.transform(words_longest_word)
    hashed_features_sld = hasher_sld.transform(words_sld)
    hashed_longest_word_df = pd.DataFrame(hashed_features_longest_word.toarray(),columns=[f'lw_hash_{i}' for i in range(hashed_features_longest_word.shape[1])])
    hashed_sld_df = pd.DataFrame(hashed_features_sld.toarray(), columns=[f'sld_hash_{i}' for i in range(hashed_features_sld.shape[1])])
    
    df.reset_index(drop=True, inplace=True)
    hashed_sld_df.reset_index(drop=True, inplace=True)
    hashed_longest_word_df.reset_index(drop=True, inplace=True)
    df = pd.concat([df, hashed_sld_df, hashed_longest_word_df], axis=1)
    
    df = df.drop(['longest_word', 'sld'], axis=1)
    features = ['FQDN_count', 'subdomain_length', 'lower', 'numeric', 'entropy',
       'special', 'labels', 'labels_max', 'labels_average', 'sld_length',
       'sld_numeric', 'longest_word_length', 'longest_word_numeric']
    
    y = df['Target Attack']
    X = df[features]
    
    
    return X, y
    
    

In [28]:
scale = StandardScaler()

In [32]:
from skmultiflow.evaluation import EvaluateHoldout
from skmultiflow.data import DataStream

ModuleNotFoundError: No module named 'skmultiflow'

In [33]:
pip install skmultiflow

ERROR: Could not find a version that satisfies the requirement skmultiflow (from versions: none)
ERROR: No matching distribution found for skmultiflow
Note: you may need to restart the kernel to use updated packages.


In [31]:
for message in consumer:
    # Deserialize your message here
    df = deserialize(message.value) 

    # Add to window data
    window_data.append(df)

    # Check if the window is filled
    if len(window_data) >= window_size:
        # Convert window data to a suitable format for model input
        window_df = pd.concat(window_data, ignore_index=True)
        
        X , y = preprocess(window_df)
        X = scale.fit_transform(X)
        
        # Predict and evaluate for static model
        
        predictions_static = static_model.predict_proba(X)
        threshold = 0.65
        y_pred_static = (predictions_static[:, 1] >= threshold).astype(int)
        accuracy_static = f1_score(y, y_pred_static)
        performance_static.append(accuracy_static)

        # Predict and evaluate for dynamic model
        predictions_dynamic = dynamic_model.predict_proba(X)
        y_pred_dynamic = (predictions_dynamic[:, 1] >= threshold).astype(int)
        accuracy_dynamic = f1_score(y, y_pred_static)
        performance_dynamic.append(accuracy_dynamic)

        # Output the window's performance
        print(f"Window Static Model Accuracy: {accuracy_static}")
        print(f"Window Dynamic Model Accuracy: {accuracy_dynamic}")
        

        # Decide if retraining is necessary for dynamic model
        if should_retrain(performance_dynamic):  # Implement this function based on your criteria
            print("Retraining dynamic model...")
            dynamic_model.fit(window_data_array, window_labels_array)
            print("Dynamic model retrained.")

        # Reset for the next window
        window_data = []
        window_labels = []

Window Static Model Accuracy: 0.03214285714285714
Window Dynamic Model Accuracy: 0.03214285714285714
